# Generator 사용 시 발생한 issue

## 파일을 write 하는 과정에서 무한 반복...

<img src="./img/gen_issue1.jpeg" width="800px" height="200px"><br>
### [상황]
1. filepath_list(제너레이터)가 돌면서 A/a.csv, A/b.csv 차례로 리턴.
2. A/a.csv 파일을 read -> A/new.csv 생성
3. A/a.csv 에서 읽은 data를 A/new.csv에 write
4. filepath_list(제너레이터)에 A/new.csv path가 추가
5. A/new.csv 파일을 read
6. A/new.csv 내에 있는 data를 그대로 다시 A/new.csv 에 write

**=> 5, 6번이 무한 반복!!!!**

In [ ]:
def get_filepath(self):
    """ 입력 디렉토리 경로 내 존재하는 file 경로 반환

    :return: file path
    """

    if os.path.isdir(self.read_path):
        for (root, _, files) in os.walk(self.read_path):
            for file in files:
                file_path = os.path.join(root, file)
                yield file_path        
    elif os.path.isfile(self.read_path):  # 파일을 입력으로 받았을 경우
        yield self.read_path
    else:
        logging.error("해당 디렉토리 및 파일이 존재하지 않습니다.")

In [ ]:
def read(self):
    """ path 내 파일을 읽어 한줄 씩 반환 generator

    :return: file contents line
    """
    filepath_list = self.get_filepath()    # file path list
    
    for filepath in filepath_list:
        print(filepath)    # new.csv 에 멈춰있는 상태
        ext = filepath.split(".")[-1]  # 확장자
        read_function = eval(f"read_{ext}")
        for line in read_function(filepath):
            yield line

In [ ]:
def write_csv(self):
    data = self.read()
    with open(self.write_path, 'w') as file:
        wr = csv.writer(file)
        for d in data:
            wr.writerow(d)

## 해결 방법

### 1. read 할 디렉토리 경로와 write할 파일 경로를 다르게 설정

### 2. get_filepath를 generator가 아닌 리스트 객체로 반환

### 결론
1번은 해당 문제를 계속 인식하며 class를 사용해야 하고 다른 사람이 이 코드를 사용했을 때 똑같은 이슈가 발생할 수 있는 가능성이 있다. 2번 방법으로 해당 문제를 해결하는 것이 더 나을 것이라 생각한다.